In [118]:
import pandas as pd
import numpy as np
from time import time
import re
import string
import socket
from glob import glob

In [119]:
country_code = "US"
print('Country:', country_code)

# Local
if 'samuel' in socket.gethostname().lower():
    path_to_data = os.path.join('../../data/classification',country_code)
# Cluster
else:
    path_to_data = os.path.join('/scratch/spf248/twitter/data/classification',country_code)
print('Path to data:',path_to_data)

Country: US
Path to data: ../../data/classification/US


# Training Set

In [128]:
# Only keep one label per worker and tweet
labels=pd.concat(
[pd.read_csv(file) for file in glob(os.path.join(path_to_data,'labeled','*','labels.csv'))]).sort_values(
by=['tweet_id','class_id','QIDWorker']).drop_duplicates(
['tweet_id','class_id','QIDWorker']).set_index(
['tweet_id','class_id','QIDWorker'])

print('# labels:', labels.shape[0])

# labels: 67425


In [138]:
counts=labels.groupby(['tweet_id','class_id'])['score'].value_counts()

In [129]:
labels

score
tweet_id            class_id QIDWorker           
278519817328279552  1        A170EDGL8ZWMSL   yes
                             A1CFPKUOCGJIM6   yes
                             A1NHTBY5YB9JH7   yes
                             A3B7TNVOISSZ2O   yes
                             AVCXJ9M71WDCB    yes
...                                           ...
1079020674377629701 5        A1JV64BL3WCK0G   yes
                             A1XLGIFFGB01EU   yes
                             A3RR85PK3AV9TU   yes
                             A3U21PUMQ6NGT2   yes
                             AI885RNRGVIFM     no

[67425 rows x 1 columns]

In [156]:
def is_labeled(x):
    # If First sequence was allocated more than once
    if x[0]>1:
        # If no other sequence
        if len(x)==1:
            return True
        else:
            # If second sequence less popular
            if x[1]<x[0]:
                return True
    return False

# Counts labels for each observation
counts=labels.groupby(['tweet_id','class_id'])['score'].value_counts().rename('count')

# Keep tweets that were labeled more than once with most popular labels strictly dominating
ids_labeled=counts.groupby(['tweet_id','class_id']).apply(list).apply(is_labeled).groupby('tweet_id').sum().where(lambda x:x==5).dropna().index
print('# labeled tweets:', len(ids_labeled))

# labeled tweets: 2483


In [160]:
# Keep most popular label sequence
labels=counts.reindex(ids_labeled,level='tweet_id').reset_index(
level='score').groupby(['tweet_id','class_id'])['score'].first().unstack()

In [161]:
labels.to_csv(os.path.join(path_to_data,'labeled','labels.csv'))

In [162]:
class2name=dict(zip([str(x) for x in range(1,6)],[
'Does this tweet indicate that the user is currently unemployed?',
'Does this tweet indicate that the user became unemployed within the last month?',
'Does this tweet indicate that the user is currently searching for a job?',
'Does this tweet indicate that the user was hired within the last month?',
'Does this tweet contain a job offer?',
]))

class2name

{'1': 'Does this tweet indicate that the user is currently unemployed?',
 '2': 'Does this tweet indicate that the user became unemployed within the last month?',
 '3': 'Does this tweet indicate that the user is currently searching for a job?',
 '4': 'Does this tweet indicate that the user was hired within the last month?',
 '5': 'Does this tweet contain a job offer?'}

In [163]:
labels

class_id,1,2,3,4,5
tweet_id,,,,,
278519817328279552,yes,no,yes,no,no
278534362910322688,no,no,yes,no,no
278988941837873152,yes,no,no,no,no
279004515271864320,no,no,no,no,no
279777877677989889,no,no,no,no,no
...,...,...,...,...,...
1072914627594514434,yes,no,no,no,no
1073295386511589377,yes,no,no,no,no
1077327152817684480,no,no,no,no,yes
